In [ ]:
using SparseArrays, LinearAlgebra, DelimitedFiles, IndexedGraphs, Plots
using IntervalUnionArithmetic
using ProgressMeter
#using Pkg
#Pkg.activate("../")
using Causality
using Graphs
using Dictionaries

# Generation of Epidemics

In [ ]:
DS=readdlm("./RealGraph/detailed_list_of_contacts_Hospital.dat_");
timescale = 20000


#DS=readdlm("./tij_SFHH.dat_");
#timescale = 7000;

In [ ]:
DS[:,1] ./= timescale
nodes = unique(DS[:,[2,3]])
T = Float64(maximum(DS[:,1]))
N = length(nodes)
G = SimpleGraph(N)
id = Dictionary(nodes, [i for i = 1:N]); 
for st = 1:size(DS,1)
    add_edge!(G,id[DS[st,2]],id[DS[st,3]]) 
end
jobs = Dictionary([i for i = 1:N],(unique([unique(DS[:,[2,4]], dims=1) ; unique(DS[:,[3,5]], dims=1)],dims=1))[:,2])
G = IndexedBiDiGraph(G)

In [ ]:
contacts = Dictionary([(i,j) for i=1:N for j=1:N if sparse(G)[i,j] != 0 ] ,[i for i = 1:nnz(G.A)])
meet_time=20/timescale
masks = [intervalUnion(∅) for g=1:nnz(G.A)]
for st = 1:size(DS,1)
    masks[contacts[(id[DS[st,2]],id[DS[st,3]])]] = masks[contacts[(id[DS[st,2]],id[DS[st,3]])]] ∪ intervalUnion(DS[st,1]-meet_time,DS[st,1])
    #symmetrization
    masks[contacts[(id[DS[st,3]],id[DS[st,2]])]] = masks[contacts[(id[DS[st,3]],id[DS[st,2]])]] ∪ intervalUnion(DS[st,1]-meet_time,DS[st,1])
end
V = [MaskedRate(UnitRate(),mask) for mask in masks];

In [ ]:
exposuretime = [(sum(DS[:,2] .== j) + sum(DS[:,3] .== j)) * meet_time for j in id.indices]
scatter([i for i = 1:N if jobs[i] == "MED"],[exposuretime[i] for i = 1:N if jobs[i] == "MED"],label = "MED",title="Total exposure time for each individual")
scatter!([i for i = 1:N if jobs[i] == "ADM"],[exposuretime[i] for i = 1:N if jobs[i] == "ADM"], label = "ADM",xlabel="ID number")
scatter!([i for i = 1:N if jobs[i] == "NUR"],[exposuretime[i] for i = 1:N if jobs[i] == "NUR"], label = "NUR",ylabel = "time")
scatter!([i for i = 1:N if jobs[i] == "PAT"],[exposuretime[i] for i = 1:N if jobs[i] == "PAT"], label = "PAT")
#savefig("HospitalStudy")

In [ ]:
getpar(pseed,autoinf::GaussianRate,inf_in::GaussianRate) = 
    [fill(pseed, 1, N);
     fill(autoinf.a, 1, N); fill(autoinf.b, 1, N); fill(autoinf.c, 1, N);
     fill(inf_in.a,   1, N); fill(inf_in.b, 1, N); fill(inf_in.c, 1, N);
    ]

getpargen(pseed, autoinf::GaussianRate, inf_out::GaussianRate) = 
    [pseed autoinf.a autoinf.b autoinf.c inf_out.a inf_out.b inf_out.c]


In [ ]:
#Initialize generation parameters
ε = 1e-10
λ = 0.05 / 20 * timescale
pseed = 1/N
autoinf = GaussianRate(0.01, T/2, 1/ε)
inf_in = GaussianRate(1.0, T/2, 1/ε)
inf_out = GaussianRate(λ, 2., 1.)

θp = getpar(pseed, autoinf, inf_in);
θpgen = getpargen(pseed, autoinf, inf_out);

In [ ]:
#Initialize min/max boundaies
pseed_min = ε
pseed_max = 1-ε
rate_min = GaussianRate(ε  ,  -T,   ε)
rate_max = GaussianRate(1/ε  ,  2T,   1/ε)

θmin = getpar(pseed_min , rate_min, rate_min);
θmax = getpar(pseed_max , rate_max, rate_max);

rate_min = GaussianRate(ε  ,  -T,   ε)
rate_max = GaussianRate(1/ε  ,  T,   T)

θgenmin = getpargen(pseed_min, rate_min, rate_min);
θgenmax = getpargen(pseed_max, rate_max, rate_max);

In [ ]:
const Igen = GenerativeSI{GaussianRate,GaussianRate} 
const Igauss = GaussianInferentialSI

In [ ]:
Mp = StochasticModel(Igen, T, θp, G, θpgen,V);
sample! = Sampler(Mp);
xtrue = zeros(N)
sample!(xtrue)
println(sum(xtrue .< T)/N)
scatter(xtrue)

In [ ]:
function Cau_infereSI(G, V, N, T, pseed, autoinf, inf_out; ε=1e-10)
    pseed_min = ε
    pseed_max = 1-ε
    rate_min = GaussianRate(ε  ,  -T,   ε)
    rate_max = GaussianRate(1/ε  ,  2T,   1/ε)

    θmin = getpar(pseed_min , rate_min, rate_min);
    θmax = getpar(pseed_max , rate_max, rate_max);

    rate_min = GaussianRate(ε  ,  -T,   ε)
    rate_max = GaussianRate(1/ε  ,  T,   T)

    θgenmin = getpargen(pseed_min, rate_min, rate_min);
    θgenmax = getpargen(pseed_max, rate_max, rate_max);
    
    const Igen = GenerativeSI{GaussianRate,GaussianRate} 
    const Igauss = GaussianInferentialSI
    
    θp2gen = getpargen(pseed, autoinf, inf_out);
    inf_in = GaussianRate(1., T/2, 3*T );
    θp2 = getpar(pseed, autoinf, inf_in);
    Mp2 = StochasticModel(Igen, T, θp2, G, θp2gen, V);
    θ = getpar(pseed, autoinf, inf_in);
    M = StochasticModel(Igauss, T, θ, G, θp2gen, V);    
    
    ProgressMeter.ijulia_behavior(:clear)
    F = descend!(Mp2, Ofalse; M=M, numsamples=1000, numiters=100, 
         θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.1),
         hyperdescender=SignDescender(0.));
    F = descend!(Mp2, Ofalse; M=M, numsamples=1000, numiters=100, 
         θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.03),
         hyperdescender=SignDescender(0.));
    statscau = prior(M, numsamples=10000);
    return statscau
end

In [17]:
nobs_max, p = 300, 1e-5
O = [(i = rand(1:N) ; ti = T*(0.2+rand())/1.2; (i,xtrue[i] < ti,ti,p)) for l=1:nobs_max]
f_rate = 1e-5
Ofalse=[rand()< f_rate ? (o[1],!o[2],o[3],f_rate) : (o[1],o[2],o[3],f_rate) for o in O]
Oav = []
for nobs in (10, 25, 40, 80, 160, 300)
    Oav = Ofalse[1:nobs]
    θp2gen = getpargen(pseed, autoinf, inf_out);
    inf_in = GaussianRate(1., T/2, 3*T );
    θp2 = getpar(pseed, autoinf, inf_in);
    Mp2 = StochasticModel(Igen, T, θp2, G, θp2gen, V);
    θ = getpar(pseed, autoinf, inf_in);
    M = StochasticModel(Igauss, T, θ, G, θp2gen, V);
end

In [12]:
#Softened model
θp2gen = getpargen(pseed, autoinf, inf_out);
inf_in = GaussianRate(1., T/2, 3*T );

θp2 = getpar(pseed, autoinf, inf_in);
Mp2 = StochasticModel(Igen, T, θp2, G, θp2gen, V);

## Causality 

In [13]:
θ = getpar(pseed, autoinf, inf_in);
M = StochasticModel(Igauss, T, θ, G, θp2gen, V);

In [ ]:

ProgressMeter.ijulia_behavior(:clear)
F = descend!(Mp2, Ofalse; M=M, numsamples=1000, numiters=100, 
         θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.1),
         hyperdescender=SignDescender(0.));
F = descend!(Mp2, Ofalse; M=M, numsamples=1000, numiters=100, 
         θmin=θmin, θmax=θmax,θgenmin=θgenmin, θgenmax=θgenmax, descender=SignDescender(0.03),
         hyperdescender=SignDescender(0.));

Progress:  88%|████████████████████████████████████▏    |  ETA: 0:00:24
  F:  113.20165281224237

In [ ]:
statscau = prior(M, numsamples=10000);

## SoftMarg

In [ ]:
stats,weights_soft = softpostnoise(Mp, Ofalse; numsamples=2 * 10^5);

## Sib

In [ ]:
using PyCall
@pyimport sib
function NonMarkovDynamicSibyl(N, T_cont, Λ, V, a, b, c, O, γ; n_steps=20, maxit = 400, tol = 1e-14)
    dt = T_cont / n_steps
    T = Int(round(T_cont / dt))
    @show T n_steps
    i = findnz(G.A)[1]
    j = findnz(G.A)[2]    
    contacts = [(i[a]-1, j[a]-1, t, 1.) for t in 1:T for a = 1:nnz(Λ.A) if (t * dt) in V[a].mask];
    obs = [[(i,-1,t) for t=1:T for i=0:N-1];
           [(i-1,s,Int(round(t/dt))) for (i,s,t,p) in O]]
    sort!(obs, lt=((i1,s1,t1),(i2,s2,t2))->(t1<t2))
    prob_sus = 0.5
    prob_seed=γ
    pseed = prob_seed / (2 - prob_seed)
    psus = prob_sus * (1 - pseed)
    g = sib.PiecewiseLinear(pycall(sib.RealParams, PyObject, [a*exp(-((t-b)/c)^2) * dt for t =1:T]))
    params = sib.Params(prob_r=sib.Exponential(mu=0), pseed=pseed, psus=psus,pautoinf=autoinf.a*dt,fp_rate=f_rate,fn_rate=f_rate, prob_i = g)
    f = sib.FactorGraph(contacts=contacts, observations=obs, params=params)
    sib.iterate(f, maxit=maxit,tol=tol)
    sib.iterate(f, maxit=maxit, damping=0.5, tol=tol)
    sib.iterate(f, maxit=maxit, damping=0.9, tol=tol)
    p_sib=[collect(n.bt) for n in f.nodes]
    m_sib = zeros(N, T)
    for i=1:N
        m_sib[i,1] = p_sib[i][1] 
        for t=2:T
            m_sib[i,t] = m_sib[i,t-1] + p_sib[i][t]
        end
    end 
    return m_sib
end

In [ ]:
a=time()
nsteps_sib = 2100
p_sib = NonMarkovDynamicSibyl(N, T, G, V, inf_out.a, inf_out.b, inf_out.c, Ofalse, 1/N; n_steps=nsteps_sib, maxit = 40, tol = 1e-4);
b=time()
println(b-a)

# Heuristic
Causality version

In [ ]:
using IntervalUnionArithmetic
T = Float64(T)
struct HeuristicSI <: SI end
maskauto = fill(intervalUnion(0., T),N)
maskinf = fill(intervalUnion(0., T),N)
θfrench = getpar(pseed, autoinf, inf_in);
Causality.individual(M::StochasticModel{HeuristicSI}, i::Int, θi = @view(M.θ[:,i]), θg = M.θgen ) = 
@views IndividualSI(θi[1], 
    MaskedRate(GaussianRate(θi[2:4]...),maskauto[i]), 
    MaskedRate(UnitRate(),maskinf[i]), 
    GaussianRate(θg[5:7]...),)

inf_start_time = T * ones(N)
for o in Ofalse
    if o[2] == 1
        inf_start_time[o[1]] = min(inf_start_time[o[1]], o[3] - 5)
        maskauto[o[1]] = maskauto[o[1]] ∩ intervalUnion(o[3]-5, T)       
        θfrench[2,o[1]] = 10^10
        θfrench[3,o[1]] = o[3] - 5
        θfrench[4,o[1]] = 100 * T
    elseif o[2] == 0
        maskinf[o[1]] = maskinf[o[1]] ∩ intervalUnion(o[3], T)
        maskauto[o[1]] = maskauto[o[1]] ∩ intervalUnion(o[3], T)
        θfrench[1,o[1]] = 1e-10
    end
end
for i=1:N
    if inf_start_time[i] != T 
        maskauto[i] = maskauto[i] ∩ intervalUnion(inf_start_time[i], T)
    end
end

Mfrench = StochasticModel(HeuristicSI, T, θfrench, G, θp2gen, V);
statsfre = prior(Mfrench,numsamples=1000);

## Metropolis Monte Carlo

In [ ]:
K = Causality.GaussMove(2.)
#stats_mh = Causality.metropolis_sampling_parallel(Mp, O, K; numsamples = 10^3,numsteps=10^3)
xmc = zeros(N)
sample!(xmc)
Mmc = StochasticModel(Igen, T, θp, G, θpgen,V);
stats_mh = Causality.metropolis_sampling_sequential(Mp, Ofalse, K; numsamples = 6 * 10^4,numsteps= 2, x = xmc);

## Marginals and ROC curve

In [ ]:
function marginal(i, t, stats)
    numsamp = size(stats,1)
    sum(stats[:,i] .< t)/numsamp
end

function reweighted_marginal(i, t, stats, weights)
    numsamp = size(stats,1)
    @assert numsamp == size(weights,1)
    weights ./= maximum(weights)
    sum(weights .* (stats[:,i] .< t))/sum(weights)
end


function tpr(xtrue, rank) 
    if sum(xtrue) == 0
        return ones(N)
    end
    return cumsum(xtrue[rank]) ./ sum(xtrue)
end

function fpr(xtrue, rank) 
    N = size(rank,1)
    if sum(xtrue) == N
       a = zeros(N) 
       a[N] = 1
       return a 
    end    
    return (range(1,N,length=N) .- cumsum(xtrue[rank])) ./ (range(1,N,length=N) .- cumsum(xtrue[rank]) )[end]
end

function ROC(xtrue, p)
    N = size(xtrue,1)
    rank = sortperm(p, rev=true)
    
    return fpr(xtrue, rank) , tpr(xtrue, rank)
end

function AUROC(ROC)
    N = size(ROC[1],1) 
    AU = 0
    for t = 1:N-1
        AU += ROC[2][t] * (ROC[1][t+1] - ROC[1][t])
    end
    return AU
end

In [ ]:
#Marginals
bins = 20
using Plots
p_cau = zeros(N,bins)
p_french = zeros(N,bins)
p_mh = zeros(N,bins)
p_soft = zeros(N,bins)
for i = 1:N
    for t = 1:bins
       p_cau[i,t] = marginal(i, t*T/bins, statscau)
       p_french[i,t] = marginal(i, t*T/bins, statsfre)
       p_mh[i,t] = marginal(i, t*T/bins, stats_mh)
       p_soft[i,t] = reweighted_marginal(i, t*T/bins, stats, weights_soft)
    end
end

In [ ]:
i = 1
round(T)
plot(LinRange(1,T,bins),p_cau[i,:], label = "cau", legend=:right)
plot!(LinRange(1,T,bins),p_french[i,:], label = "french")
plot!(LinRange(1,T,bins),p_soft[i,:], label = "soft")
plot!(LinRange(1,T,bins),p_mh[i,:], label = "MH")
plot!(LinRange(1,T,nsteps_sib),p_sib[i,:], label = "sib")

In [ ]:
#AUROC curves
AU_curve=zeros(bins)
AU_sib=zeros(bins)
AU_french = zeros(bins)
AU_soft = zeros(bins)
AU_MH = zeros(bins)
for t = 1:bins
    cau_risk=zeros(N)
    sib_risk = zeros(N)
    french_risk = zeros(N)
    MH_risk = zeros(N)
    soft_risk = zeros(N)
    for i=1:N
       cau_risk[i] = p_cau[i,t]
       french_risk[i] = p_french[i,t]
       MH_risk[i] = p_mh[i,t]
       sib_risk[i] = p_sib[i,Int(round(t/bins*nsteps_sib))]
       soft_risk[i] = p_soft[i,t]
    end
    xt = xtrue .< t * T / bins
    AU_soft[t] = AUROC(ROC(xt, soft_risk))
    AU_curve[t] = AUROC(ROC(xt, cau_risk))
    AU_french[t] = AUROC(ROC(xt, french_risk))
    AU_MH[t] = AUROC(ROC(xt, MH_risk))
    AU_sib[t] = AUROC(ROC(xt, sib_risk))
end

plot(LinRange(1,T,bins),AU_curve, label="AUCau", title="Causality VS Sib AUROC in function of time")
scatter!(LinRange(1,T,bins),AU_soft, label="AUSoft",legend=:bottomleft)
scatter!(LinRange(1,T,bins),AU_french, label="AUFrench")
scatter!(LinRange(1,T,bins),AU_MH, label="AUMH")
scatter!(LinRange(1,T,bins), AU_sib, label="AUsib")
xlabel!("t")
ylabel!("AUROC")
#savefig("confronti.pdf")
#ylims!(0.5,1.01)

In [82]:
trial = 1

1

In [83]:
open("./RealGraph/try$(trial)nobs$(nobs)cau.txt","a") do io
    writedlm(io,AU_curve') 
end
open("./RealGraph/try$(trial)nobs$(nobs)sib.txt","a") do io
    writedlm(io,AU_sib')  
end
open("./RealGraph/try$(trial)nobs$(nobs)fre.txt","a") do io
    writedlm(io,AU_french') 
end
open("./RealGraph/try$(trial)nobs$(nobs)mc.txt","a") do io
    writedlm(io,AU_MH') 
end
open("./RealGraph/try$(trial)nobs$(nobs)soft.txt","a") do io
    writedlm(io,AU_soft') 
end